In [7]:
import os
import json
import pandas as pd
import traceback
import tqdm as notebook_tqdm



In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
Key = os.getenv("GOOGLE_API_KEY")

In [14]:
llm = ChatGoogleGenerativeAI(google_api_key=Key,model="gemini-pro",temperature=0.5)

In [35]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [52]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [53]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs. DO not include RESPONSE_JSON in the output and make sure output is valid JSON format
### RESPONSE_JSON
{response_json}

"""

In [54]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [55]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [56]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [57]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [58]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [59]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)

In [60]:
filePath = r"/Users/sanchitsetia/mcqgen/file.txt"

In [61]:
with open(filePath, 'r') as file:
    TEXT = file.read()


In [62]:
NUMBER=5 
SUBJECT="Deep learning"
TONE="simple"

In [63]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Deep learning is the subset of machine learning methods based on artificial neural networks (ANNs) with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]
Deep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]
Artificial neural networks were inspired by information processing and distributed communication nodes in bio

In [64]:
response

{'text': 'Deep learning is the subset of machine learning methods based on artificial neural networks (ANNs) with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]\nDeep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]\nArtificial neural networks were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains. Specifically, artificial 

In [65]:
quiz = response.get('quiz')

In [66]:
quiz

'{"1": {"mcq": "What does the term \'deep\' in \'deep learning\' refer to?", "options": {"a": "The number of layers in the network", "b": "The complexity of the algorithms used", "c": "The amount of data used for training", "d": "The speed at which the network can learn"}, "correct": "a"}, "2": {"mcq": "Which of the following is NOT a type of deep learning architecture?", "options": {"a": "Deep neural networks", "b": "Recurrent neural networks", "c": "Support vector machines", "d": "Convolutional neural networks"}, "correct": "c"}, "3": {"mcq": "What is the main benefit of using deep learning for unsupervised learning tasks?", "options": {"a": "It eliminates the need for labeled data", "b": "It can extract more complex features from the data", "c": "It is more efficient than supervised learning", "d": "It is more accurate than supervised learning"}, "correct": "a"}, "4": {"mcq": "Which of the following is a key advantage of integrated photonics for deep learning hardware?", "options": 

In [67]:
quiz=json.loads(quiz)

In [68]:
quiz

{'1': {'mcq': "What does the term 'deep' in 'deep learning' refer to?",
  'options': {'a': 'The number of layers in the network',
   'b': 'The complexity of the algorithms used',
   'c': 'The amount of data used for training',
   'd': 'The speed at which the network can learn'},
  'correct': 'a'},
 '2': {'mcq': 'Which of the following is NOT a type of deep learning architecture?',
  'options': {'a': 'Deep neural networks',
   'b': 'Recurrent neural networks',
   'c': 'Support vector machines',
   'd': 'Convolutional neural networks'},
  'correct': 'c'},
 '3': {'mcq': 'What is the main benefit of using deep learning for unsupervised learning tasks?',
  'options': {'a': 'It eliminates the need for labeled data',
   'b': 'It can extract more complex features from the data',
   'c': 'It is more efficient than supervised learning',
   'd': 'It is more accurate than supervised learning'},
  'correct': 'a'},
 '4': {'mcq': 'Which of the following is a key advantage of integrated photonics for 

In [69]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [70]:
quiz_table_data

[{'MCQ': "What does the term 'deep' in 'deep learning' refer to?",
  'Choices': 'a: The number of layers in the network | b: The complexity of the algorithms used | c: The amount of data used for training | d: The speed at which the network can learn',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a type of deep learning architecture?',
  'Choices': 'a: Deep neural networks | b: Recurrent neural networks | c: Support vector machines | d: Convolutional neural networks',
  'Correct': 'c'},
 {'MCQ': 'What is the main benefit of using deep learning for unsupervised learning tasks?',
  'Choices': 'a: It eliminates the need for labeled data | b: It can extract more complex features from the data | c: It is more efficient than supervised learning | d: It is more accurate than supervised learning',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is a key advantage of integrated photonics for deep learning hardware?',
  'Choices': 'a: Massively parallel data transfer | b: Low p

In [71]:
quiz=pd.DataFrame(quiz_table_data)

In [72]:
quiz

,MCQ,Choices,Correct
0,What does the term 'deep' in 'deep learning' r...,a: The number of layers in the network | b: Th...,a
1,Which of the following is NOT a type of deep l...,a: Deep neural networks | b: Recurrent neural ...,c
2,What is the main benefit of using deep learnin...,a: It eliminates the need for labeled data | b...,a
3,Which of the following is a key advantage of i...,a: Massively parallel data transfer | b: Low p...,a
4,What is the main difference between artificial...,a: ANNs are static while biological brains are...,a


In [73]:
quiz.to_csv("deeplearning.csv",index=False)